In [48]:
#import libraries

import numpy as np
import pandas as pd

#data stuff
#import data with pandas
df = pd.read_csv('crisis_dataset.csv')

#data cleaning

# drop notes column, drop complexity, drop A1 Nukes and A2 Nukes, drop name
df.drop(['NOTES','COMPLEXITY','A1 NUKES','A2 NUKES','NAME'], axis=1, inplace=True)

# drop the rows that are completely empty
df.dropna(how='all', inplace=True)

#dropna??
#df.dropna(inplace=True)

#iterate through crisis, history, row for each crisis
first_column = 'CRISNO'
second_column = 'HISTORY'

unique_crisis_no = df[first_column].unique()
final_selected_rows = []

print(df)
print(unique_crisis_no)

# iterate over crisis nos
for first_value in unique_crisis_no:
    # this filters data for unique crisis no
    filtered_df_first = df[df[first_column] == first_value]
    
    # this initializes the unique history for each crisis
    unique_history = filtered_df_first[second_column].unique()
    chosen_history = np.random.choice(unique_history) # choose one unique history randomly

    final_filtered_df = filtered_df_first[filtered_df_first[second_column] == chosen_history] # filter data for the chosen history

    # randomly select a single row from filtered data
    final_selected_row = final_filtered_df.sample(n=1)

    # append to final list
    final_selected_rows.append(final_selected_row)

# combine to df
final_combined_df = pd.concat(final_selected_rows)

print(final_combined_df)

#model stuff






     CRISNO  YEAR  HISTORY ACTOR 1 ACTOR 2            A1 THREAT  \
2     445.0  2004      1.0     GRG     RUS         CONVENTIONAL   
3     446.0  2005      1.0     ETH     ERI         CONVENTIONAL   
4     447.0  2005      1.0     CHA     SUD         CONVENTIONAL   
5     448.0  2006      1.0     IRN     USA                  NaN   
6     448.0  2006      2.0     IRN     UKG              NUCLEAR   
..      ...   ...      ...     ...     ...                  ...   
991   171.0  1959      NaN     CHN     IND         CONVENTIONAL   
992   172.0  1959      NaN     IRQ     IRN  CONVENTIONAL, OTHER   
993   173.0  1960      NaN     EGY     ISR         CONVENTIONAL   
994   175.0  1960      NaN     DOM     VEN         CONVENTIONAL   
995   175.0  1960      NaN     DOM     USA                  NaN   

               A2 THREAT  A1 INTENSITY  A2 INTENSITY  KINETIC  ...  \
2           CONVENTIONAL           2.0           1.0      1.0  ...   
3           CONVENTIONAL           3.0           3.0   

ValueError: a must be greater than 0 unless no samples are taken